# What happens:
- It makes the check and saves the result in an excel
- Almost always empty, so you just add the row according to (https://rcrs.rbinternational.com/wiki/display/OCR/WRONG-WAY-RISK+Manual+Corrections)
- If not, just follow the link above for the other case

## Import 

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32
from IPython.lib.display import Audio

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [ ]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

### Make connection with Oracle DB

In [ ]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### Also see how much time it took

In [ ]:
%%time
d = pd.read_sql_query("""select 'BOUGHT CDS' as product_type,
cds1.contract_reference as contract_reference,
cds1.fermat_attribute_5 as product_id,
'' as repo_type,
cds1.cds_ccp_gcc_code as gcc_ctpy, 
g1.entity_desc as gcc_ctpy_name,
cds1.reference_issuer_Gcc_code as gcc_issuer, 
g2.entity_desc as gcc_issuer_name,
cds1.reference_isin as security_reference,
sum(cds1.ead_pre_ccf) as exposure
from
(select	
	case	
		when cd.long_short = 'S' then 'protection_sold'
		when cd.long_short = 'L' then 'protection_bought'
		else 'ERROR' end cds_position,
	cd.book_code, cd.contract_reference, cd.counterparty cds_cpty, e2.entity_desc cds_cpty_name,  rzb_cdr2.gcc_head_code cds_ccp_gcc_code, rzb_cdr2.gcc_head_name cds_ccp_gcc_name,cd.market_value, cd.currency, cd.nominal, cd.value_date, cd.maturity_date,	
	cd.contract_reference fermat_cds_id, cd.attribute_5 fermat_attribute_5, s.security_reference reference_isin,	
	s.issuer_code reference_issuer_code, e.entity_desc reference_issuer_name, rzb_cdr1.gcc_head_code reference_issuer_Gcc_code, rzb_cdr1.gcc_head_name reference_issuer_Gcc_name ,  t.ead_pre_ccf, t.rwa	
	from credit_derivative cd	
		left join entity e2 on cd.counterparty = e2.entity_code
		left join reference_list rl on cd.basket_reference = rl.basket_reference
		left join security s on s.security_reference = rl.contract_reference
		left join entity e on s.issuer_code = e.entity_code
		left join t_cdr t on cd.attribute_5 = t.exposure_attribute_5 
		left join rzb_cdr rzb_cdr1 on s.issuer_code = rzb_cdr1.counterparty_code 
		left join rzb_cdr rzb_cdr2 on cd.counterparty  = rzb_cdr2.counterparty_code 
	where cd.contract_type = 'IS1000')cds1	
left join entity g1 on cds1.cds_ccp_gcc_code = g1.entity_code		
left join entity g2 on cds1.reference_issuer_Gcc_code = g2.entity_code		
    where cds1.cds_position like '%bought'		
and cds1.cds_ccp_gcc_code =  cds1.reference_issuer_Gcc_code		
group by cds1.contract_reference, cds1.cds_ccp_gcc_code, cds1.reference_issuer_Gcc_code, cds1.reference_isin,cds1.fermat_Attribute_5, g1.entity_Desc, g2.entity_desc		
UNION ALL		
SELECT  'REPO' as product_type,		
rep1.contract_reference, 		
rep1.exposure_attribute_5 as product_id, 		
rep1.repo_type as repo_type,             		
rep1.repo_ccp_gcc_head_code as gcc_ctpy, 		
g3.entity_desc as gcc_ctpy_name,		
rep1.issuer_gcc_head_code as gcc_issuer, 		
g4.entity_desc as gcc_issuer_name,		
rep1.security_reference, 		
rep1.EAD_PRE_CCF_Agg as exposure		
FROM 		
(                 		
 select sub01.counterparty_original as Repo_Ctpy,   e1.entity_group as repo_ccp_Gcc_head_code,           		
 e1.description as Repo_Ctpy_Name,                		
 e1.incorporation_country as Repo_Ctpy_Risk_Country,                		
 e1.entity_type  as Repo_Ctpy_type,                		
 sub01.internal_rating_original as Counterparty_Rating,                		
 sub01.netting_agreement_reference,                 		
 sub01.table_name,                 		
 sub01.contract_type,                		
 ct.ct_desc_user,                		
 sub01.dealbook,                		
 sub01.booking_company,                		
 sub01.exposure_attribute_5,                		
 r.contract_reference,                 		
 r.repo_type,                 		
 s.security_reference,                		
 s.security_type,                		
 e2.entity_code as Issuer,                		
 e2.description as Issuer_Name,                		
e2.entity_group as issuer_Gcc_head_code,           		
 e2.incorporation_country as Issuer_Risk_Country,                		
 e2.entity_type  as Issuer_type,                		
 e2.actual_entity_class as Issuer_Type_Desc,                		
 case                		
  when sub01.table_name='REPO_NETTING_VOL' then sub01.EAD_PRE_CCF_Agg               		
  when r.repo_type = 'R' then rp.securities_market_value*c2.exch_rate               		
  when r.repo_type = 'RR' then r.cash_nominal*c1.exch_rate               		
  else Null               		
 end as exposure_in_EUR,                		
 case  when r.repo_type = 'R' then r.cash_nominal*c1.exch_rate               		
  when r.repo_type = 'RR' then rp.securities_market_value*c2.exch_rate               		
  else Null               		
 end as collateral_in_EUR,                		
 sub01.Collateralised,                		
 sub01.Uncollateralised,                		
 sub01.EAD_PRE_CCF_Agg,                		
 sub01.RWA_Agg                		
 from                 		
  (select t.counterparty_original,                		
  t.netting_agreement_reference,                		
  t.dealbook,               		
  t.booking_company,               		
  t.contract_type,               		
  t.table_name,                		
  t.internal_rating_original,               		
  t.exposure_attribute_5,               		
  t.contract_Reference,               		
  sum(case when t.tranche_collateral_type like 'COLL%' then t.ead_pre_ccf               		
   end) as Collateralised,              		
  sum(case when t.tranche_collateral_type not like 'COLL%' then t.ead_pre_ccf               		
   end) as Uncollateralised,              		
  sum(t.ead_pre_ccf) as EAD_PRE_CCF_Agg,               		
  sum(t.rwa) as RWA_Agg               		
  from t_cdr t               		
  group by                		
  t.contract_reference,               		
  t.booking_company,               		
  t.contract_type,               		
  t.exposure_attribute_5,               		
  t.internal_rating_original,               		
  t.dealbook,               		
  t.counterparty_original,                		
  t.netting_agreement_reference,                		
  t.table_name               		
  ) sub01               		
 left join repo r on sub01.contract_reference = r.contract_reference                		
 left join currency c1 on r.cash_currency = c1.ccy_code                		
 left join repo_portfolio rp on r.contract_reference = rp.repo_reference                		
 left join currency c2 on rp.ccy_code = c2.ccy_code                		
 left join contract_types ct on sub01.contract_type = ct.ct_code_user                		
 left join esr e1 on sub01.counterparty_original = e1.entity_code                		
 left join security s on rp.security_reference = s.security_reference                		
 left join esr e2 on s.issuer_code = e2.entity_code                		
 left join entity_type et2 on e2.et_code_user = et2.entity_type                		
 where sub01.table_name like 'REP%'                		
 and sub01.dealbook in ('HO_B','HO_T','RZBA_B')		
 and r.repo_Type = 'RR'                		
 UNION                		
select sub01.counterparty_original as Repo_Ctpy, e1.entity_group as repo_ccp_Gcc_head_code,                 		
 e1.description as Repo_Ctpy_Name,                		
 e1.incorporation_country as Repo_Ctpy_Risk_Country,                		
 e1.entity_type  as Repo_Ctpy_type,                		
 sub01.internal_rating_original as Counterparty_Rating,                		
 sub01.netting_agreement_reference,                 		
 sub01.table_name,                 		
 sub01.contract_type,                		
 ct.ct_desc_user,                		
 sub01.dealbook,                		
 sub01.booking_company,                		
 sub01.exposure_attribute_5,                		
 r.contract_reference,                 		
 r.repo_type,                 		
 case when r.repo_type = 'R' then s2.security_reference                		
 when r.repo_type = 'RR' then s3.security_reference end as security_reference,                		
 case when r.repo_type = 'R' then s2.security_type                		
 when r.repo_type = 'RR' then s3.security_type end as security_type,                		
 case when r.repo_type = 'R' then e2.entity_code                		
 when r.repo_type = 'RR' then e3.entity_code end as Issuer,                		
 case when r.repo_type = 'R' then e2.description                		
 when r.repo_type = 'RR' then e3.description end as Issuer_Name,                		
 case when r.repo_type = 'R' then e2.entity_group          		
 when r.repo_type = 'RR' then e3.entity_group end as issuer_Gcc_head_code,         		
 case when r.repo_type = 'R' then e2.country_code                		
when r.repo_type = 'RR' then e3.country_code end as Issuer_Risk_Country,                		
 case when r.repo_type = 'R' then e2.et_code_user                		
 when r.repo_type = 'RR' then e3.et_code_user end as Issuer_type,                		
 case when r.repo_type = 'R' then et2.entity_type_desc                		
 when r.repo_type = 'RR' then et3.entity_type_desc end as Issuer_Type_Desc,                		
 case                		
  when r.repo_type = 'R' then rp.securities_market_value*c2.exch_rate               		
  when r.repo_type = 'RR' then r.cash_nominal*c1.exch_rate               		
  else Null               		
 end as exposure_in_EUR,                		
 case  when r.repo_type = 'R' then c.market_value*c3.exch_rate               		
  when r.repo_type = 'RR' then c.market_value*c3.exch_rate               		
  else Null               		
 end as collateral_in_EUR,                		
 sub01.Collateralised,                		
 sub01.Uncollateralised,                		
 sub01.EAD_PRE_CCF_Agg,                		
 sub01.RWA_Agg                		
 from                 		
  (         		
select t.counterparty_original,                		
  t.netting_agreement_reference,                		
  t.dealbook,               		
  t.booking_company,               		
  t.contract_type,               		
  t.internal_rating_original,               		
  t.table_name,                		
  t.exposure_attribute_5,               		
  t.contract_Reference,               		
 sum(case when t.tranche_collateral_type = 'COLLATERALIZED' then t.ead_pre_ccf               		
   end) as Collateralised,              		
  sum(case when t.tranche_collateral_type ='NOT COLLATERALIZED' then t.ead_pre_ccf               		
   end) as Uncollateralised,            		
  sum(t.ead_pre_ccf) as EAD_PRE_CCF_Agg,               		
  sum(t.rwa) as RWA_Agg               		
  from t_cdr t               		
  where t.table_name like 'REP%'                		
  group by                		
  t.contract_reference,               		
  t.contract_type,               		
  t.exposure_attribute_5,               		
  t.internal_rating_original,               		
  t.dealbook,               		
  t.booking_company,               		
  t.counterparty_original,                		
  t.netting_agreement_reference,                		
  t.table_name               		
  ) sub01               		
 left join repo r on sub01.contract_reference = r.contract_reference                		
 left join currency c1 on r.cash_currency = c1.ccy_code                		
 left join repo_portfolio rp on r.contract_reference = rp.repo_reference                		
 left join currency c2 on rp.ccy_code = c2.ccy_code                		
 left join contract_types ct on sub01.contract_type = ct.ct_code_user                		
 left join esr e1 on sub01.counterparty_original = e1.entity_code                		
 left join security s2 on rp.security_reference = s2.security_reference                		
 left join esr e2 on s2.issuer_code = e2.entity_code                		
 left join entity_type et2 on e2.et_code_user = et2.entity_type                		
 left join contract_guarantee cg on r.contract_reference = cg.benef_contract_ref                		
 left join collateral c on cg.guar_contract_ref = c.contract_reference                		
 left join currency c3 on c.market_value_ccy = c3.ccy_code                		
 left join security s3 on c.security_reference = s3.security_reference                		
 left join esr e3 on s3.issuer_code = e3.entity_code                		
 left join entity_type et3 on e3.et_code_user = et3.entity_type                		
 where sub01.table_name like 'REP%'                		
 and sub01.dealbook not in  ('HO_B','HO_T','RZBA_B','RZBA_T') 		
and r.repo_type = 'RR'          		
 )rep1  		
left join entity g3 on rep1.repo_ccp_gcc_head_code = g3.entity_code		
left join entity g4 on rep1.issuer_gcc_head_code = g4.entity_code		
where rep1.repo_ccp_gcc_head_code = rep1.issuer_gcc_head_code		
and rep1.contract_type not in ('IL0101')		
and (rep1.collateralised is not null and rep1.collateralised > 0)		
""", connection_RAY)

### Save var in an excel file

In [ ]:
# determining the name of the file
file_name = rd + '_Wrong_way_risk.xlsx'
ex_folder = "Export/"

# saving the excel
d.to_excel(ex_folder + file_name, index=False)
print('DataFrame is written to Excel File successfully.')